# Moving and adapting

In [1]:
from dune.grid import reader
from dune.mmesh import mmesh
from dune.fem.view import adaptiveLeafGridView as adaptive

dim = 2
gridView = adaptive( mmesh((reader.gmsh, "vertical.msh"), dim) )

hgrid = gridView.hierarchicalGrid
igridView = hgrid.interfaceGrid

In [1]:
from ufl import *
from dune.ufl import Constant

t = Constant(0, name="t")
tEnd = 1
dt = 0.1

def speed():
    return as_vector([1.0, 0.0])

def movement(x):
    return as_vector([1.0, 0.0])

def f(u):
    return speed() * u

def u0(x):
    return conditional(x[0] < 0.5, 1.0, 0.0)

def uexact(x, t):
    return u0( x - t * speed() )

In [2]:
from dune.fem.space import finiteVolume

space = finiteVolume(gridView)
u = TrialFunction(space)
v = TestFunction(space)

x = SpatialCoordinate(space)
n = FacetNormal(space)

uh = space.interpolate(u0(x), name="uh")
uh_old = uh.copy()

NameError: name 'gridView' is not defined

In [ ]:
import numpy as np
from dune.geometry import vertex
from dune.mmesh import edgeMovement

def getShifts():
    mapper = igridView.mapper({vertex: 1})
    shifts = np.zeros((mapper.size, dim))
    for v in igridView.vertices:
        shifts[ mapper.index(v) ] = as_vector(movement( v.geometry.center ))
    return shifts

em = edgeMovement(gridView, getShifts())

def g(u, n):
    sgn = inner(speed(), n('+'))
    return inner( conditional( sgn > 0, f( u('+') ), f( u('-') ) ), n('+') )

def gBnd(u, n):
    sgn = inner(speed(), n)
    return inner( conditional( sgn > 0, f(u), f(uexact(x, t)) ), n )

def h(u, n):
    sgn = inner(em('+'), n('+'))
    return conditional( sgn > 0, sgn * u('+'), sgn * u('-') )

In [ ]:
from dune.fem.scheme import galerkin

detPsi = abs(det(nabla_grad(x + dt * em)))

a = (u - uh_old * detPsi) * v * dx
a += dt * (g(u, n) - h(u, n)) * jump(v) * dS
a += dt * gBnd(u, n) * v * ds

scheme = galerkin([a == 0], solver=("suitesparse","umfpack"))

In [ ]:
from dune.fem import adapt
from dune.fem.plotting import plotPointData as plot

for step in range(1, int(tEnd/dt)+1):
  
    hgrid.ensureInterfaceMovement(getShifts()*dt)
    hgrid.markElements()
    adapt([uh])

    hgrid.moveInterface(getShifts()*dt)

    em = edgeMovement(gridView, getShifts())
    t += dt

    uh_old.assign(uh)
    scheme.solve(target=uh)
    
    plot(uh)